In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interpolate

from DarkCapPy import *
import DarkCapPy.DarkPhoton as DP

import matplotlib.pylab as plt
import seaborn as sns

from scipy.interpolate import griddata
from skimage import measure

import os

In [ ]:
DP.Planet_Mass

In [ ]:
session_name = os.getenv('JPY_SESSION_NAME')

print(session_name)

jpy_name = os.path.basename(session_name)

print(jpy_name)

In [ ]:
#infilename = 'rates_muons_alpha_therm_one_month.parquet'
#infilename = 'rates_muons_alpha_max_one_month.parquet'
#infilename = 'rates_muons_alpha_max_one_month_ALL_MASSES.parquet'
infilename = 'rates_muons_electrons_both_alphas_ALL_MASSES.parquet'


df = pd.read_parquet(infilename)
df

In [ ]:
filter = df['rate_CMS_1yr'] > 50

df[filter]

In [ ]:
df.plot.scatter(x='mx', y='rate_CMS_1yr')

In [ ]:
df_max_rates = None
for mx_test in np.arange(1000, 11000, 1000):
    print(mx_test)
    filter = (df['mx']==mx_test) & (df['rate_CMS_1yr'] > 10)
    
    high_rate = df[filter]['rate_CMS_1yr'].max()
    print(high_rate)

    filter = filter & (df['rate_CMS_1yr']==high_rate)

    if df is None:
        df_max_rates = df[filter].copy()
    else:
        df_max_rates = pd.concat([df_max_rates, df[filter].copy()])

In [ ]:
df['alpha_therm_or_max'].value_counts()

In [ ]:
df_max_rates

In [ ]:
df_max_rates.to_csv('maximum_rates_at_CMS.csv')

In [ ]:
#rate = 10*12*df_results['rate']
#rate = 10*12*df_results['rate_CMS']

#rate = df_results['rate_CMS_1month']

#final_state_particles = 'muons'
final_state_particles = 'electrons'

alphax_assumption = 'MAX'
#alphax_assumption = 'THERMAL'

live_time = '10yrs'
#live_time = '1month'
#live_time = '1yr'

rate_detector = 'rate'
#rate_detector = 'rate_CMS'

filter =          (df['final_state_particles']==final_state_particles)
filter = filter & (df['alpha_therm_or_max']==alphax_assumption)

df_tmp = df[filter]

rate_string = f"{rate_detector}_{live_time}"
print(f"Plotting for rate {rate_string}")
rate = df_tmp[rate_string]
mx_vals = df_tmp['mx']

x = df_tmp['ma']
y = df_tmp['epsilon']

#plt.figure(figsize=(16,15))
plt.figure(figsize=(10,9))


mxs = [10, 100, 1000, 10000]
#mxs = [100, 1000, 1000, 10000]

#mxs = [1000, 10000, 100000, 1000000]

for idx,test_mx in enumerate(mxs):
#for idx,test_mx in enumerate([10, 100, 1000, 10000]):
#for idx,test_mx in enumerate([1000, 10000, 100000, 1000000]):

    print(f"{idx} {test_mx}")

    plt.subplot(2,2,idx+1)
    
    for expected_rate in [1, 10, 100, 1000]:

        # For Icecube
        fractional_test = 0.3
        # For CMS
        #fractional_test = 1.0
        
        filter = (test_mx == mx_vals) & (np.abs(rate-expected_rate)/expected_rate<fractional_test)
        #filter = filter & (df_results['final_state_particles']=='electrons')
        #filter = filter & (df_results['alpha_therm_or_max']=='MAX')

        #x = df_tmp['ma']
        #y = df_tmp['epsilon']
        
        plt.plot(x[filter], y[filter], '.', label=f'rate={expected_rate}')

        #print(test_mx, expected_rate, df_results[filter]['rate_CMS'])

    plt.xlim(0.01, 10)
    plt.ylim(1e-11, 1e-6)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel(r"$m_{A'}$ [GeV]", fontsize=18)
    plt.ylabel(r"$\epsilon$", fontsize=18)

    plt.legend()

    plt.title(f'$M_X = {int(test_mx):d}$ GeV', fontsize=18)
#plt.legend()
plt.tight_layout()

#plt.savefig("Fig3_comparison_10_years_IceCube_electrons.png")
#plt.savefig("Fig3_comparison_10_years_IceCube_muons.png")
#plt.savefig("Fig3_comparison_1_month_CMS_electrons.png")

#plt.savefig("Fig3_comparison_1_month_CMS_muons_alpha_therm.png")

#plt.savefig("Fig3_comparison_1_month_CMS_muons_alpha_max_high_masses.png")
file_string = f"Fig3_comparison_{rate_string}_{final_state_particles}_alphax_{alphax_assumption}_mX_range_{mxs[0]}_{mxs[-1]}.png"
plt.savefig(file_string)

In [ ]:
#final_state_particles = 'muons'
final_state_particles = 'electrons'
alphax_assumption = 'MAX'
#alphax_assumption = 'THERMAL'
live_time = '10yrs'
#live_time = '1month'
#live_time = '1yr'
rate_detector = 'rate'
#rate_detector = 'rate_CMS'

filter =          (df['final_state_particles']==final_state_particles)
filter = filter & (df['alpha_therm_or_max']==alphax_assumption)

df_tmp = df[filter]

rate_string = f"{rate_detector}_{live_time}"
print(f"Plotting for rate {rate_string}")

mx_vals = df_tmp['mx']

# Step 1: Example data
# Assume you have irregularly spaced data
#x = np.random.uniform(0, 10, 1000)
#y = np.random.uniform(0, 10, 1000)
#z = np.sin(x) * np.cos(y)  # or your actual data

plt.figure(figsize=(10,9))

for idx,test_mx in enumerate(mxs):
#for idx,test_mx in enumerate([10, 100, 1000, 10000]):
#for idx,test_mx in enumerate([1000, 10000, 100000, 1000000]):

    print(f"{idx} {test_mx}")

    plt.subplot(2,2,idx+1)

    #test_mx = 1000
    filter = (test_mx == df_tmp['mx'])
    
    x = df_tmp['ma'][filter]
    y = df_tmp['epsilon'][filter]
    z = df_tmp[rate_string][filter]
    
    
    # Step 2: Interpolate onto a regular grid
    xi = np.linspace(min(x), max(x), 1000)
    yi = np.linspace(min(y), max(y), 1000)
    xi, yi = np.meshgrid(xi, yi)
    zi = griddata((x, y), z, (xi, yi), method='nearest')
        
    colors=['blue', 'orange', 'green', 'red']
    
    for jdx,nevents in enumerate([1, 10, 100, 1000]):
        # Step 3: Extract the contour at desired height
        z0 = nevents  # for example
        contours = measure.find_contours(zi, level=z0)
        
        #print(contours)
        
        # Convert contour coordinates back to real x-y scale
        contour_coords = []
        for contour in contours:
            x_coords = xi[0, :]
            y_coords = yi[:, 0]
            real_x = np.interp(contour[:, 1], np.arange(len(x_coords)), x_coords)
            real_y = np.interp(contour[:, 0], np.arange(len(y_coords)), y_coords)
            contour_coords.append(np.column_stack((real_x, real_y)))
        
        # Now contour_coords is a list of (x, y) points where z ≈ z0
        
        # Optional: Plot to verify
        plt.contour(xi, yi, zi, levels=[z0], colors=colors[jdx])#, colors='red')
        #plt.scatter(x, y, c=z, cmap='viridis', s=1)

        # Dummy plot for the legend
        plt.plot([1e6, 1e6], [2e6, 3e6], '-', color=colors[jdx], label=f'rate={nevents}')
    
    
    
    plt.title(f'Contour where M$_X$ = {test_mx}')
    #plt.colorbar(label='z')
    
    
    plt.xlim(0.01, 10)
    plt.ylim(1e-11, 1e-6)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel(r"$m_{A'}$ [GeV]", fontsize=18)
    plt.ylabel(r"$\epsilon$", fontsize=18)
    plt.legend()

plt.tight_layout()

In [ ]:
print(df['epsilon'].unique())

print(len(df['epsilon'].unique()))

In [ ]:
df.info()

In [ ]:
# Vary the parameters

final_state_particles = 'muons'
#final_state_particles = 'electrons'

alphax_assumption = 'MAX'
#live_time = '10yrs'
live_time = '1month'

#rate_detector = 'rate'
rate_detector = 'rate_CMS'

filter = (df['final_state_particles']==final_state_particles)
filter = filter & (df['alpha_therm_or_max']==alphax_assumption)

df_tmp = df[filter]

rate_string = f"{rate_detector}_{live_time}"

print(f"Plotting for rate {rate_string}")
#rate = df_tmp[rate_string]

mx_vals = df_tmp['mx']

x = df_tmp['ma']
y = df_tmp[rate_string]

plt.figure(figsize=(16,15))


#mxs = [10, 100, 1000, 10000]
mxs = [1000, 10000, 100000, 1000000]
#mxs = df_tmp['mx'].unique()

epsilons = df_tmp['epsilon'].unique()
epsilons = epsilons[18:]

for idx,eps in enumerate(epsilons):

    #print(f"{idx} {eps}")

    plt.subplot(6,3,idx+1)
    
    for mx in mxs:
        
        filter = (df_tmp['mx']== mx) & (df_tmp['epsilon']==eps)
        #filter = filter & (df_results['final_state_particles']=='electrons')
        #filter = filter & (df_results['alpha_therm_or_max']=='MAX')

        #x = df_tmp['ma']
        #y = df_tmp['epsilon']
        
        plt.plot(x[filter], y[filter], '.', label=f'mX={int(mx):d}')

        #print(test_mx, expected_rate, df_results[filter]['rate_CMS'])

    if final_state_particles == 'muons':
        plt.xlim(0.2, 1)
        plt.ylim(1,2000)
    elif final_state_particles == 'electrons':
        plt.xlim(0.01, 10)
        plt.ylim(1,2000000)

    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel(r"$m_{A'}$ [GeV]", fontsize=18)
    plt.ylabel(r"rate", fontsize=18)

    plt.legend()

    plt.title(f'$\epsilon = {eps:.0e}$', fontsize=18)
#plt.legend()
plt.tight_layout()

#plt.savefig("Fig3_comparison_10_years_IceCube_electrons.png")
#plt.savefig("Fig3_comparison_10_years_IceCube_muons.png")
#plt.savefig("Fig3_comparison_1_month_CMS_electrons.png")

#plt.savefig("Fig3_comparison_1_month_CMS_muons_alpha_therm.png")

#plt.savefig("Fig3_comparison_1_month_CMS_muons_alpha_max_high_masses.png")
file_string = f"Rates_vs_mx_{rate_string}_{final_state_particles}_alphax_{alphax_assumption}_mX_range_{mxs[0]}_{mxs[-1]}.png"
plt.savefig(file_string)

In [ ]:
ma_vals = df_tmp['ma']
print(len(ma_vals))

ma_vals


def find_index_of_closest_value(values_to_look_for, arr):

    closest_values = []
    indices = []
    
    for v in values_to_look_for:
        diff = np.abs(arr - v)
        
        min_diff = min(diff)
        
        idx = diff.to_list().index(min_diff)
        
        print(idx, min_diff, arr.iloc[idx])

        closest_values.append(arr.iloc[idx])
        indices.append(idx)

    return indices, closest_values

vals = [0.22, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.6]
indices, closest_vals = find_index_of_closest_value(vals, ma_vals)
print(indices, closest_vals)

In [ ]:
# Vary the parameters

final_state_particles = 'muons'
#final_state_particles = 'electrons'

alphax_assumption = 'MAX'
#live_time = '10yrs'
live_time = '1month'

#rate_detector = 'rate'
rate_detector = 'rate_CMS'

filter = (df['final_state_particles']==final_state_particles)
filter = filter & (df['alpha_therm_or_max']==alphax_assumption)

df_tmp = df[filter]

rate_string = f"{rate_detector}_{live_time}"

print(f"Plotting for rate {rate_string}")
#rate = df_tmp[rate_string]

ma_vals = df_tmp['ma']

x = df_tmp['mx']
y = df_tmp[rate_string]

plt.figure(figsize=(16,15))


#mxs = [10, 100, 1000, 10000]
#mxs = [1000, 10000, 100000, 1000000]
#mxs = df_tmp['mx'].unique()

mas = ma_vals.unique()
print(len(mas), mas)

# Find index

#mas = [mas[500], mas[550], mas[600], mas[650],mas[700],mas[750]]
#print(mas)
vals = [0.22, 0.24, 0.26, 0.28, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55]
indices, closest_vals = find_index_of_closest_value(vals, ma_vals)
mas = closest_vals


#mas = [0.23, 0.3, 0.4, 0.5]


epsilons = df_tmp['epsilon'].unique()
epsilons = epsilons[18:]

for idx,eps in enumerate(epsilons):

    #print(f"{idx} {eps}")

    plt.subplot(6,3,idx+1)
    
    for ma in mas:
        
        filter = (df_tmp['ma']== ma) & (df_tmp['epsilon']==eps)
        #filter = filter & (df_results['final_state_particles']=='electrons')
        #filter = filter & (df_results['alpha_therm_or_max']=='MAX')

        #x = df_tmp['ma']
        #y = df_tmp['epsilon']

        #print(x[filter].unique())
        
        plt.plot(x[filter], y[filter], '.-', label=f'ma={ma:.2f}')

        #print(test_mx, expected_rate, df_results[filter]['rate_CMS'])

    if final_state_particles == 'muons':
        plt.xlim(500)
        plt.ylim(1, 2000)
    elif final_state_particles == 'electrons':
        plt.xlim(1)
        plt.ylim(1)

    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel(r"$m_{X}$ [GeV]", fontsize=18)
    plt.ylabel(r"rate", fontsize=18)

    plt.legend()

    plt.title(f'$\epsilon = {eps:.0e}$', fontsize=18)
#plt.legend()
plt.tight_layout()

#plt.savefig("Fig3_comparison_10_years_IceCube_electrons.png")
#plt.savefig("Fig3_comparison_10_years_IceCube_muons.png")
#plt.savefig("Fig3_comparison_1_month_CMS_electrons.png")

#plt.savefig("Fig3_comparison_1_month_CMS_muons_alpha_therm.png")

#plt.savefig("Fig3_comparison_1_month_CMS_muons_alpha_max_high_masses.png")
file_string = f"Rates_vs_ma_{rate_string}_{final_state_particles}_alphax_{alphax_assumption}_mX_range_{mxs[0]}_{mxs[-1]}.png"
plt.savefig(file_string)

# Energy loss stuff

From ChatGPT but need to verify


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants for standard rock
rho = 2.65  # g/cm^3
a = 2.0e-3  # GeV·cm^2/g (ionization loss)
b = 4.0e-6  # cm^2/g (radiative loss coefficient)



def energy_loss(E0, x, rho=2.65, a=2e-3, b=4e-6):
    """
    Calculate the remaining energy E(x) of a muon after traversing distance x in cm.
    E0: initial energy in GeV
    x: distance in cm
    """
    X = rho * x  # Convert distance to g/cm^2
    epsilon = a / b
    return (E0 + epsilon) * np.exp(-b * X) - epsilon

def muon_range(E0, rho=2.65, a=2e-3, b=4e-6):
    """
    Calculate the range R of a muon with initial energy E0 in cm.
    E0: initial energy in GeV
    """
    epsilon = a / b
    X = (1 / b) * np.log(1 + E0 / epsilon)  # Range in g/cm^2
    return X / rho  # Convert to cm

# Example usage
E0_values = [10, 100, 1000, 10000]  # GeV
for E0 in E0_values:
    R = muon_range(E0)
    print(f"Initial Energy: {E0} GeV -> Range: {R/1000:.2f} m")

# Plotting energy loss over distance for a 100 GeV muon
x_vals = np.linspace(0, 5000, 500)  # Distance in cm
E_vals = energy_loss(100, x_vals, rho)

plt.figure(figsize=(10, 6))
plt.plot(x_vals / 100, E_vals)  # Convert cm to m for x-axis
plt.xlabel('Distance Traversed (m)')
plt.ylabel('Muon Energy (GeV)')
plt.title('Muon Energy Loss in Standard Rock (Initial Energy = 100 GeV)')
plt.grid(True)
plt.show()


In [ ]:
# Constants for carbon
rho = 2.267  # g/cm^3
a = 1.9e-3  # GeV·cm^2/g (ionization loss)
b = 0.9e-6  # cm^2/g (radiative loss coefficient)


# Amount lost per 1cm
#E_i = np.arange(1,100000,1)
E_i = np.arange(1,1000,1)

E_vals = energy_loss(E_i, np.ones_like(E_i), rho, a, b)

dE = 1000*(E_i - E_vals)/rho

# To try to replicate the plot in the PDG, maybe divide by the density?
#dE /= rho

plt.figure(figsize=(10, 6))
plt.plot(E_i, dE)  # Convert cm to m for x-axis
plt.xlabel('Energy')
plt.ylabel('dE')
#plt.ylim(1,4000)
plt.ylim(1, 10)
plt.yscale('log')
plt.xscale('log')
plt.title('Muon Energy Loss in Standard Rock (Initial Energy = 100 GeV)')
plt.grid(True)
plt.show()

In [ ]:
# Copper 
rho = 9.0  # g/cm^3
a = 1.75e-3  # GeV·cm^2/g (ionization loss)
b = 1.40e-5  # cm^2/g (radiative loss coefficient)

# Plotting energy loss over distance for a 100 GeV muon
x_vals = np.linspace(0, 500000, 500)  # Distance in cm

E_i = 10000
E_vals = energy_loss(E_i, x_vals, rho, a, b)


plt.figure(figsize=(10, 6))
plt.plot(x_vals / 100, E_vals)  # Convert cm to m for x-axis
plt.xlabel('Distance Traversed (m)')
plt.ylabel('Muon Energy (GeV)')
plt.title('Muon Energy Loss in Standard Rock (Initial Energy = 100 GeV)')
plt.grid(True)
plt.show()

In [ ]:
# Amount lost per 1cm
E_i = np.arange(1,100000,1)
E_vals = energy_loss(E_i, np.ones_like(E_i), rho, a, b)

dE = 1000*(E_i - E_vals)/rho

# To try to replicate the plot in the PDG, maybe divide by the density?
#dE /= rho

plt.figure(figsize=(10, 6))
plt.plot(E_i, dE)  # Convert cm to m for x-axis
plt.xlabel('Energy')
plt.ylabel('dE')
plt.ylim(1,4000)
#plt.ylim(0.001, 400)
plt.yscale('log')
plt.xscale('log')
plt.title('Muon Energy Loss in Standard Rock (Initial Energy = 100 GeV)')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
rho_rock = 2.5  # g/cm^3
a_ion = 2.0e-3  # GeV·cm^2/g, average ionization loss for hadrons
dEdx_ion = a_ion * rho_rock  # GeV/cm

# Hadronic interaction lengths in rock
lambda_pi = 120  # g/cm^2
lambda_K = 160   # g/cm^2
lambda_cm_pi = lambda_pi / rho_rock  # cm
lambda_cm_K = lambda_K / rho_rock    # cm

def simulate_hadron(E0, particle='pion', distance=5000):
    """
    Simulate deterministic energy loss of a charged pion or kaon through rock.
    
    Parameters:
    - E0: initial energy in GeV
    - particle: 'pion' or 'kaon'
    - distance: total distance in cm
    
    Returns:
    - x_vals: array of distances in cm
    - E_vals: array of energies in GeV at each step
    """
    if particle == 'pion':
        lambda_cm = lambda_cm_pi
    elif particle == 'kaon':
        lambda_cm = lambda_cm_K
    else:
        raise ValueError("Unsupported particle type. Use 'pion' or 'kaon'.")

    # Parameters
    f = 0.6  # Fraction of energy retained after each hadronic interaction

    # Initialize
    step_size = 10  # cm
    steps = int(distance / step_size)
    x_vals = np.linspace(0, distance, steps + 1)
    E_vals = [E0]

    E = E0
    dist_since_last_interaction = 0

    for i in range(1, steps + 1):
        # Ionization energy loss
        E -= dEdx_ion * step_size
        dist_since_last_interaction += step_size

        # Check for hadronic interaction
        if dist_since_last_interaction >= lambda_cm:
            E *= f
            dist_since_last_interaction = 0

        E = max(E, 0)
        E_vals.append(E)

        if E <= 0:
            # Stop tracking after particle ranges out
            x_vals = x_vals[:i+1]
            break

    return x_vals, E_vals

# Run simulations
x_pi, E_pi = simulate_hadron(100, 'pion', distance=10000)
x_K, E_K = simulate_hadron(100, 'kaon', distance=10000)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(np.array(x_pi)/100, E_pi, label='Charged Pion (100 GeV)')
plt.plot(np.array(x_K)/100, E_K, label='Charged Kaon (100 GeV)', linestyle='--')
plt.xlabel('Distance Traversed in Rock (m)')
plt.ylabel('Remaining Energy (GeV)')
plt.title('Deterministic Energy Loss of Charged Pion and Kaon in Rock')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
def simulate_hadron_range(E0, particle='pion'):
    """
    Simulate deterministic energy loss of a charged pion or kaon through rock
    until the particle runs out of energy.

    Parameters:
    - E0: initial energy in GeV
    - particle: 'pion' or 'kaon'

    Returns:
    - x_vals: array of distances in cm
    - E_vals: array of energies in GeV at each step
    - total_range: final range in cm
    """
    if particle == 'pion':
        lambda_cm = lambda_cm_pi
    elif particle == 'kaon':
        lambda_cm = lambda_cm_K
    else:
        raise ValueError("Unsupported particle type. Use 'pion' or 'kaon'.")

    f = 0.6  # Energy retained after interaction
    step_size = 10  # cm

    x_vals = [0]
    E_vals = [E0]
    E = E0
    x = 0
    dist_since_last_interaction = 0

    while E > 0:
        # Ionization energy loss
        E -= dEdx_ion * step_size
        dist_since_last_interaction += step_size
        x += step_size

        # Hadronic interaction check
        if dist_since_last_interaction >= lambda_cm:
            E *= f
            dist_since_last_interaction = 0

        E = max(E, 0)
        x_vals.append(x)
        E_vals.append(E)

    return x_vals, E_vals, x  # x is total range in cm


In [ ]:
# Simulate and print range for 100 GeV pion and kaon
x_pi, E_pi, R_pi = simulate_hadron_range(100, 'pion')
x_K, E_K, R_K = simulate_hadron_range(100, 'kaon')

print(f"Pion range: {R_pi/100:.2f} m")
print(f"Kaon range: {R_K/100:.2f} m")

# Optional: plot the energy loss
plt.figure(figsize=(10, 6))
plt.plot(np.array(x_pi)/100, E_pi, label='Charged Pion (100 GeV)')
plt.plot(np.array(x_K)/100, E_K, label='Charged Kaon (100 GeV)', linestyle='--')
plt.xlabel('Distance Traversed in Rock (m)')
plt.ylabel('Remaining Energy (GeV)')
plt.title('Deterministic Pion and Kaon Range in Rock')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

def energy_loss(E0, particle, material, distance_cm=None):
    """
    Vectorized version. E0 can be a float or a numpy array of energies in GeV.
    
    Parameters:
    - E0: float or np.ndarray, initial energy in GeV
    - particle: 'muon', 'pion', or 'kaon'
    - material: one of 'rock', 'copper', 'lead', 'water', 'aluminum', 'iron', 'carbon', 'ice'
    - distance_cm: optional float. If None, return range in cm; else return final energy in GeV

    Returns:
    - If distance_cm is None: range(s) in cm
    - If distance_cm is provided: final energy/energies in GeV
    """
    particle = particle.lower()
    material = material.lower()
    E0 = np.asarray(E0)  # support scalar or array

    material_db = {
        "rock":     {"rho": 2.65,  "a_mu": 2.0e-3, "b_mu": 4.0e-6,  "lambda_pi": 120, "lambda_K": 160},
        "copper":   {"rho": 8.96,  "a_mu": 1.75e-3, "b_mu": 1.4e-5, "lambda_pi": 106, "lambda_K": 130},
        "lead":     {"rho": 11.34, "a_mu": 1.5e-3, "b_mu": 5.4e-5, "lambda_pi": 106, "lambda_K": 130},
        "water":    {"rho": 1.0,   "a_mu": 2.0e-3, "b_mu": 3.0e-6,  "lambda_pi": 120, "lambda_K": 160},
        "aluminum": {"rho": 2.7,   "a_mu": 1.9e-3, "b_mu": 6.5e-6,  "lambda_pi": 118, "lambda_K": 155},
        "iron":     {"rho": 7.87,  "a_mu": 1.75e-3, "b_mu": 1.7e-5, "lambda_pi": 109, "lambda_K": 135},
        "carbon":   {"rho": 2.267, "a_mu": 1.9e-3, "b_mu": 0.9e-6,  "lambda_pi": 120, "lambda_K": 160},
        "ice":      {"rho": 0.917, "a_mu": 2.0e-3, "b_mu": 3.0e-6,  "lambda_pi": 120, "lambda_K": 160},
    }

    if material not in material_db:
        raise ValueError(f"Unsupported material '{material}'.")

    mat = material_db[material]
    rho = mat["rho"]

    if particle == "muon":
        a_cm = mat["a_mu"] * rho
        b_cm = mat["b_mu"] * rho

        if distance_cm is None:
            if b_cm > 0:
                return (1 / b_cm) * np.log(1 + (b_cm * E0 / a_cm)), rho
            else:
                return E0 / a_cm, rho
        else:
            if b_cm > 0:
                epsilon = a_cm / b_cm
                return np.maximum((E0 + epsilon) * np.exp(-b_cm * distance_cm) - epsilon, 0), rho
            else:
                return np.maximum(E0 - a_cm * distance_cm, 0), rho

    elif particle in ["pion", "kaon"]:
        a_ion = 2.0e-3
        dEdx_ion = a_ion * rho
        f = 0.6
        lambda_gcm2 = mat["lambda_pi"] if particle == "pion" else mat["lambda_K"]
        lambda_cm = lambda_gcm2 / rho

        def simulate_range_or_final_energy(E_start):
            E = E_start
            x = 0
            dist_since_int = 0
            step = 1  # cm
            if distance_cm is None:
                while E > 0:
                    E -= dEdx_ion * step
                    dist_since_int += step
                    x += step
                    if dist_since_int >= lambda_cm:
                        E *= f
                        dist_since_int = 0
                #print("HERE", x, rho, type(x), type(rho))
                return x, rho
            else:
                while x < distance_cm and E > 0:
                    E -= dEdx_ion * step
                    dist_since_int += step
                    x += step
                    if dist_since_int >= lambda_cm:
                        E *= f
                        dist_since_int = 0
                #print('THERE', max(E,0), rho)
                return max(E, 0), rho

        # Vectorize the above simulation for numpy arrays
        return np.vectorize(simulate_range_or_final_energy)(E0)#, rho

    else:
        raise ValueError("Unsupported particle. Choose 'muon', 'pion', or 'kaon'.")

# Example usage
examples = [
    {"E0": 100, "particle": "muon", "material": "rock"},
    {"E0": 100, "particle": "muon", "material": "rock", "distance_cm": 5000},
    {"E0": 100, "particle": "pion", "material": "rock"},
    {"E0": 100, "particle": "pion", "material": "rock", "distance_cm": 5000},
    {"E0": 1000, "particle": "muon", "material": "ice"},
    {"E0": 1000, "particle": "kaon", "material": "copper"},
]

example_results = []
for ex in examples:
    print("----------")
    result, rho = energy_loss(**ex)
    print("outside")
    print(result)
    print(type(result))
    entry = ex.copy()
    if "distance_cm" in ex:
        entry["final_energy (GeV)"] = result
    else:
        entry["range (cm)"] = result
        entry["range (m)"] = result / 100
    example_results.append(entry)

#tools.displ
example_results

In [ ]:
E0 = np.linspace(1,10000,100)
particle = 'muon'
#particle = 'pion'

#material = 'copper'
material = 'rock'
#efinal = energy_loss(E0, particle, material, distance_cm=None)
efinal, rho = energy_loss(E0, particle, material, distance_cm=1)

print(E0 - efinal)

plt.plot(E0,  (E0-efinal)*1000/rho)
plt.xscale('log')
plt.yscale('log')
plt.ylim(1, 500)



# Test of contours

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from skimage import measure

# Step 1: Example data
# Assume you have irregularly spaced data
x = np.random.uniform(0, 10, 1000)
y = np.random.uniform(0, 10, 1000)
z = np.sin(x) * np.cos(y)  # or your actual data

# Step 2: Interpolate onto a regular grid
xi = np.linspace(min(x), max(x), 500)
yi = np.linspace(min(y), max(y), 500)
xi, yi = np.meshgrid(xi, yi)
zi = griddata((x, y), z, (xi, yi), method='cubic')

# Step 3: Extract the contour at desired height
z0 = 0.5  # for example
contours = measure.find_contours(zi, level=z0)

# Convert contour coordinates back to real x-y scale
contour_coords = []
for contour in contours:
    x_coords = xi[0, :]
    y_coords = yi[:, 0]
    real_x = np.interp(contour[:, 1], np.arange(len(x_coords)), x_coords)
    real_y = np.interp(contour[:, 0], np.arange(len(y_coords)), y_coords)
    contour_coords.append(np.column_stack((real_x, real_y)))

# Now contour_coords is a list of (x, y) points where z ≈ z0

# Optional: Plot to verify
plt.figure()
plt.contour(xi, yi, zi, levels=[z0], colors='red')
plt.scatter(x, y, c=z, cmap='viridis', s=1)
plt.title(f'Contour where z = {z0}')
plt.colorbar(label='z')
plt.show()
